In [1]:
import pandas as pd
import math
import numpy as np
import warnings
import re
import csv
import os
import xml.etree.ElementTree as ET
# import junitparser
from junitparser import Error, Failure, JUnitXml, TestCase, TestSuite, Properties
import xml.etree.ElementTree as ET
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import datetime
sys.setrecursionlimit(200000) 
print(sys.getrecursionlimit())

200000


In [2]:
full_lists = []
def access_modifier(num):
    switch_dict = {0: "Package",1: "Public",2: "Private",4: "Protected"}
    return switch_dict.get(num, "Unknown")
def process_list(lst):
    result = []
    extracted_lists = []
    i = 0
    while i < len(lst):
        if isinstance(lst[i], list):
            extracted_lists.append(lst[i])
            i += 1
        else:
            result.append(lst[i])
            i += 1
    return result, extracted_lists

def process_and_append(extracted_list, full_lists):
    levels, extracted_lists = process_list(extracted_list)
    full_lists.append(levels)
    for extracted_sublist in extracted_lists:
        process_and_append(extracted_sublist, full_lists)
        
def contains_list(lst):
    for item in lst:
        if isinstance(item, list):
            return True
    return False

def process_section(lines, index):
    try:
        result = []
        while index < len(lines):
            line = lines[index]
            index += 1
            if line.startswith("Start test"):
                sub_section, index = process_section(lines, index)
                result.append([line] + sub_section)
            elif line.startswith("End test"):
                result.append(line)
                return result, index
            else:
                result.append(line)
        return result, index
    except Exception as e:
        return e

def process_methods_section(lines, index):
    try:
        result = []
        while index < len(lines):
            line = lines[index]
            method = ""
            index += 1
            if line.startswith("Start method call:") or line.startswith("Start constructor call:"):
                method = line.split(" call: ")[1]
                sub_section, index = process_methods_section(lines, index)
                result.append([line] + sub_section)
            elif line.startswith("End method call:") or line.startswith("End constructor call:"):
                # if method == line.split(" call: ")[1]:
                    # print("Method:",method)
                    # print("Method2:",line.split(" call: ")[1])
                    result.append(line)
                    return result, index 
            else:
                result.append(line)
        return result, index
    except Exception as e:
        return e

def start_parsing(log, testcaseName, fileName):
    try:
        full_lists = []
        lines = log
        result = []
        index = 0
        sections, index = process_section(lines, index)
        for section in sections:
            result.append(section)
        process_and_append(result, full_lists)
        full_lists = [x for x in full_lists if x]
        invoked = []
        for full_stacks in full_lists:
            if len(full_stacks) > 0:
                test_name = full_stacks[0]
                resultsTmp, indexTmp = process_methods_section(full_stacks,0)
                for resultTmp in resultsTmp:
                    if type(resultTmp) == list:
                        invoked_methods = [x for x in resultTmp if not isinstance(x,list)]
                        if len(invoked_methods) > 1 and len(test_name) > 0  and not invoked_methods[0].startswith("Start constructor"):
                            end = invoked_methods[len(invoked_methods)-1].split(": ")[1]
                            start = invoked_methods[0].split(": ")[1]
                            if start == end:
                                invoked.append([fileName.split("/")[0],fileName,testcaseName,test_name.split(": ")[1].split(" ")[len(test_name.split(": ")[1].split(" "))-1],
                                    (access_modifier(int(invoked_methods[0].split(": ")[1].split(" ")[0])%8)),int(invoked_methods[0].split(": ")[1].split(" ")[0]),
                                    invoked_methods[0].split(": ")[1].split(" ")[1]])
                            else:
                                raise Exception()
        return invoked
    except:
        print(testcaseName,"on",fileName)
    # for invoke in invoked:
        # print(invoke)
        

In [ ]:
surefire_log_paths = []
with open('surefire-reports-path.tsv', 'r') as file:
    for line in file:
        surefire_log_paths.append(line.strip())  # Append each line to the list, removing leading/trailing whitespace

all_result = []
for surefire_log_path in surefire_log_paths:
    # print("(",datetime.datetime.now(),surefire_log_path,end="")
    try:
        tree = ET.parse(surefire_log_path)
        root = tree.getroot()
        for testcase in root.findall('.//testcase'):
            testcaseName = testcase.get('classname') + "." + testcase.get('name') + "()"
            # print("TESTCASENAME",testcaseName)
            if testcase.find('failure') is not None or testcase.find('error') is not None or testcase.find('skipped') is not None or len(testcase.findall('.//system-err')) > 0 or len(testcase.findall('.//system-out')) == 0:
                continue
            # testcaseName = testcase.get('classname') + "." + testcase.get('name') + "()"
            for sysout in testcase.findall('.//system-out'):
                lines = sysout.text.strip().split('\n')
                start_strings = ["Start method","End method", "Start test", "End test", "Start constructor", "End constructor"]

                substring_found = False

                for string in lines:
                    if not any(string.startswith(start) for start in start_strings):
                        substring_found = True
                        break  # Exit the loop if the substring is found

                if substring_found is False:
                    filtered_lines = [line for line in lines if any(line.strip().startswith(start) for start in start_strings)]
                    fileName = surefire_log_path.replace("/Volumes/2TB_EXT/projects_tmp/","").split("/target/surefire-reports")[0]
                    rslt = start_parsing(filtered_lines, testcaseName,fileName)
                    if rslt is not None:
                        tsv_string = '\n'.join('\t'.join(map(str, row)) for row in rslt)
                        with open('final_zzz12345.tsv', 'a') as tsv_file:
                            tsv_file.write(tsv_string + '\n')
        # print(datetime.datetime.now(), end=")")
    except ET.ParseError as e:
        print(surefire_log_path)
        continue
    except FileNotFoundError:
        print(surefire_log_path)
        continue
    except:
        print("fail",surefire_log_path)
        continue

cn.leancloud.kafka.consumer.FetcherTest.testPauseAndPartialResume() on leancloud-kafka-java-consumer
cn.leancloud.kafka.consumer.FetcherTest.testCloseConsumerFailed() on leancloud-kafka-java-consumer
